In [ ]:
!pip install semantic-link-labs

#### Imports
----------

In [2]:
import sempy.fabric as fabric
import sempy_labs as labs
from pyspark.sql.functions import *

StatementMeta(, 1b672b6a-efd4-4e97-b4c2-1566588a1514, 9, Finished, Available, Finished)

#### References
----------

In [3]:
class TestSemanticModel:
    """
    A class used to represent a test plan on a semantic model

    ...

    Attributes
    ----------
    dataset_name : str
        Name of the dataset to test.

    Methods
    -------
    test_many_to_many_relationships()
        Checks if the dataset has many to many relationships.
    test_relationship_violations()
        Checks whether the relationships in the data set follow the referential integrity.
    test_model_best_practices()
        Applies Best Practice Analyzer to the dataset.
    """

    def __init__(self, dataset_name:str):
        """
        Parameters
        ----------
        dataset_name : str
            Name of the dataset.
        """
        self.dataset_name = dataset_name
    
    def test_mtom_relationships(self):
        """
        Checks if the dataset has many to many relationships.

        Raises
        -------
        AssertionError
            If result not equals to expected value.
        """

        df = fabric.list_relationships(self.dataset_name)
        df = spark.createDataFrame(df)

        mm_relationships = df.where(col("Multiplicity") == "m:m")

        try:
            assert mm_relationships.count() == 0, f"\033[1;31mFAILED\033[0m\n\033[31m" + "".join([f"\tFound a m:m relationship between {r['From Table']}[{r['From Column']}] and {r['To Table']}[{r['To Column']}]\n" for r in mm_relationships.collect()]) + "\033[0m"
        except AssertionError as e:
            print(e)
        else:
            print("\033[1;32mPASSED\033[0m")


    def test_relationship_violations(self):
        """
        Checks whether the relationships in the data set follow the referential integrity.

        Raises
        -------
        AssertionError
            If result not equals to expected value.
        """

        tables = {table[1]: fabric.read_table(self.dataset_name, table[1]) for table in fabric.list_tables(self.dataset_name)["Name"].items()}
        relationship_violations = fabric.list_relationship_violations(tables)
        relationship_violations = spark.createDataFrame(relationship_violations).select("Message")

        try:
            assert relationship_violations.count() == 0, f"\033[1;31mFAILED\033[0m\n\033[31m" + "".join([f"\t{r['Message']}\n" for r in relationship_violations.collect()]) + "\033[0m"
        except AssertionError as e:
            print(e)
        else:
            print("\033[1;32mPASSED\033[0m")
    
    def test_model_best_practices(self):
        """
        Applies Best Practice Analyzer to the dataset.
        """

        labs.run_model_bpa(self.dataset_name)


StatementMeta(, 1b672b6a-efd4-4e97-b4c2-1566588a1514, 10, Finished, Available, Finished)

In [4]:
print("\nInitializing test session...")
test_dataset = TestSemanticModel("Contoso")

print("========================================= test session starts =========================================\n")

print("------------------------------ TEST 01 - test_many_to_many_relationships ------------------------------\n")
test_dataset.test_mtom_relationships()

print("------------------------------- TEST 02 - test_relationship_violations --------------------------------\n")
test_dataset.test_relationship_violations()

print("--------------------------------- TEST 03 - test_model_best_practices ---------------------------------\n")
test_dataset.test_model_best_practices()

print("=======================================================================================================\n")

StatementMeta(, 1b672b6a-efd4-4e97-b4c2-1566588a1514, 11, Finished, Available, Finished)


Initializing test session...
========================================= test session starts =========================================

------------------------------ TEST 01 - test_many_to_many_relationships ------------------------------

FAILED
	Found a m:m relationship between Orders[ProductKey] and Product[ProductKey]
	Found a m:m relationship between Orders[StoreKey] and Store[StoreKey]
	Found a m:m relationship between Sales[ProductKey] and Product[ProductKey]
	Found a m:m relationship between Sales[StoreKey] and Store[StoreKey]

------------------------------- TEST 02 - test_relationship_violations --------------------------------

FAILED
	1 out of 58 values in Orders[StoreKey] not present in Store[StoreKey]: 10
	4 out of 2517 values in Orders[ProductKey] not present in Product[ProductKey]: 2230,23,593,1921
	1 out of 58 values in Sales[StoreKey] not present in Store[StoreKey]: 10
	4 out of 2517 values in Sales[ProductKey] not present in Product[ProductKey]: 2230,23,593,1921

---

Rule Name,Object Type,Object Name,Severity
Add data category for columnsAdd Data Category property for appropriate columns.,Column,'Customer'[City],ℹ️
Add data category for columnsAdd Data Category property for appropriate columns.,Column,'Customer'[Continent],ℹ️
Add data category for columnsAdd Data Category property for appropriate columns.,Column,'Customer'[CountryFull],ℹ️
Add data category for columnsAdd Data Category property for appropriate columns.,Column,'Customer'[Country],ℹ️
Add data category for columnsAdd Data Category property for appropriate columns.,Column,'Customer'[Latitude],ℹ️
Add data category for columnsAdd Data Category property for appropriate columns.,Column,'Customer'[Longitude],ℹ️
Add data category for columnsAdd Data Category property for appropriate columns.,Column,'Store'[CountryCode],ℹ️
Add data category for columnsAdd Data Category property for appropriate columns.,Column,'Store'[CountryName],ℹ️
"Do not summarize numeric columnsNumeric columns (integer, decimal, double) should have their SummarizeBy property set to ""None"" to avoid accidental summation in Power BI (create measures instead).",Column,'CurrencyExchange'[Exchange],⚠️
"Do not summarize numeric columnsNumeric columns (integer, decimal, double) should have their SummarizeBy property set to ""None"" to avoid accidental summation in Power BI (create measures instead).",Column,'Customer'[Age],⚠️
